## Встраивания

В предыдущем примере мы работали с высокоразмерными векторами мешка слов длиной `vocab_size` и явно преобразовывали низкоразмерные вектора позиционного представления в разреженное представление с одним активным элементом. Такое представление с одним активным элементом неэффективно с точки зрения памяти. Кроме того, каждое слово рассматривается независимо от других, поэтому закодированные таким образом вектора не отражают семантические сходства между словами.

В этом разделе мы продолжим изучать набор данных **News AG**. Для начала загрузим данные и возьмем некоторые определения из предыдущего раздела.


In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

### Что такое эмбеддинг?

Идея **эмбеддинга** заключается в представлении слов с помощью низкоразмерных плотных векторов, которые отражают семантическое значение слова. Позже мы обсудим, как создавать осмысленные эмбеддинги слов, но пока просто будем рассматривать эмбеддинги как способ уменьшения размерности вектора слова.

Таким образом, слой эмбеддинга принимает слово на вход и выдает выходной вектор заданного размера `embedding_size`. В некотором смысле, он очень похож на слой `Dense`, но вместо того, чтобы принимать вектор в формате one-hot encoding, он может принимать номер слова.

Используя слой эмбеддинга в качестве первого слоя нашей сети, мы можем перейти от модели "мешка слов" к модели **мешка эмбеддингов**, где сначала каждое слово в тексте преобразуется в соответствующий эмбеддинг, а затем вычисляется некоторая агрегирующая функция для всех этих эмбеддингов, например, `sum`, `average` или `max`.

![Изображение, показывающее классификатор с использованием эмбеддингов для пяти слов последовательности.](../../../../../translated_images/ru/embedding-classifier-example.b77f021a7ee67eee.webp)

Наша нейронная сеть-классификатор состоит из следующих слоев:

* Слой `TextVectorization`, который принимает строку на вход и преобразует ее в тензор с номерами токенов. Мы зададим разумный размер словаря `vocab_size` и будем игнорировать менее часто используемые слова. Входная форма будет равна 1, а выходная форма — $n$, так как мы получим $n$ токенов в результате, каждый из которых содержит числа от 0 до `vocab_size`.
* Слой `Embedding`, который принимает $n$ чисел и преобразует каждое число в плотный вектор заданной длины (в нашем примере 100). Таким образом, входной тензор формы $n$ будет преобразован в тензор формы $n\times 100$.
* Агрегирующий слой, который вычисляет среднее значение этого тензора вдоль первой оси, то есть вычисляет среднее всех $n$ входных тензоров, соответствующих разным словам. Для реализации этого слоя мы будем использовать слой `Lambda` и передадим в него функцию для вычисления среднего. Выходная форма будет равна 100, и это будет численное представление всей входной последовательности.
* Финальный линейный классификатор `Dense`.


In [3]:
vocab_size = 30000
batch_size = 128

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,    
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 100)         3000000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 3,000,404
Trainable params: 3,000,404
Non-trainable params: 0
_________________________________________________________________


В выводе `summary`, в колонке **output shape**, первое измерение тензора `None` соответствует размеру минибатча, а второе — длине последовательности токенов. Все последовательности токенов в минибатче имеют разные длины. О том, как с этим справляться, мы поговорим в следующем разделе.

Теперь давайте обучим сеть:


In [4]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 20s 20ms/step - loss: 0.7891 - acc: 0.8155 - val_loss: 0.4470 - val_acc: 0.8642


> **Примечание**: мы создаем векторизатор на основе подмножества данных. Это делается для ускорения процесса, и это может привести к ситуации, когда не все токены из нашего текста присутствуют в словаре. В этом случае эти токены будут игнорироваться, что может немного снизить точность. Однако в реальной жизни подмножество текста часто дает хорошую оценку словаря.


### Работа с переменными размерами последовательностей

Давайте разберемся, как происходит обучение на минибатчах. В приведенном выше примере входной тензор имеет размерность 1, и мы используем минибатчи длиной 128, так что фактический размер тензора составляет $128 \times 1$. Однако количество токенов в каждом предложении различается. Если мы применим слой `TextVectorization` к одному входу, количество возвращаемых токенов будет различным, в зависимости от того, как текст был токенизирован:


In [5]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


Однако, когда мы применяем векторизатор к нескольким последовательностям, он должен создавать тензор прямоугольной формы, поэтому он заполняет неиспользуемые элементы токеном PAD (который в нашем случае равен нулю):


In [6]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]], dtype=int64)>

Здесь мы можем видеть встраивания:


In [7]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 1.53059261e-02,  6.80514947e-02,  3.14026810e-02, ...,
         -8.92002955e-02,  1.52911525e-04, -5.65562584e-02],
        [ 2.57456154e-01,  2.79364467e-01, -2.03605562e-01, ...,
         -2.07474351e-01,  8.31158683e-02, -2.03911960e-01],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02]],

       [[ 1.89674050e-01,  2.61548996e-01, -3.67433839e-02, ...,
         -2.07366899e-01, -1.05442435e-01, -2.36952081e-01],
        [ 6.16133213e-02,  1.80511594e-01,  9.77298319e-02, ...,
         -5.46628237e-02, -1.07340455e-01, -1.06589

> **Примечание**: Чтобы минимизировать количество заполнения, в некоторых случаях имеет смысл отсортировать все последовательности в наборе данных в порядке увеличения длины (или, точнее, количества токенов). Это обеспечит, что каждая минипартия будет содержать последовательности схожей длины.


## Семантические эмбеддинги: Word2Vec

В нашем предыдущем примере слой эмбеддинга обучался отображать слова в векторные представления, однако эти представления не имели семантического значения. Было бы здорово обучить векторное представление таким образом, чтобы похожие слова или синонимы соответствовали векторам, которые находятся близко друг к другу с точки зрения некоторого векторного расстояния (например, евклидова расстояния).

Для этого нам нужно предварительно обучить модель эмбеддинга на большом наборе текстов, используя такую технику, как [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Она основана на двух основных архитектурах, которые используются для создания распределенного представления слов:

 - **Непрерывная модель "мешок слов"** (Continuous bag-of-words, CBoW), где мы обучаем модель предсказывать слово по окружающему контексту. Учитывая n-грамму $(W_{-2},W_{-1},W_0,W_1,W_2)$, цель модели — предсказать $W_0$ на основе $(W_{-2},W_{-1},W_1,W_2)$.
 - **Непрерывная модель "пропускающий грамм"** (Continuous skip-gram) является противоположностью CBoW. Модель использует окружающее окно контекстных слов для предсказания текущего слова.

CBoW работает быстрее, а skip-gram, хотя и медленнее, лучше справляется с представлением редких слов.

![Изображение, показывающее алгоритмы CBoW и Skip-Gram для преобразования слов в векторы.](../../../../../translated_images/ru/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Чтобы поэкспериментировать с эмбеддингом Word2Vec, предварительно обученным на наборе данных Google News, мы можем использовать библиотеку **gensim**. Ниже мы находим слова, наиболее похожие на 'neural'.

> **Note:** Когда вы впервые создаете векторные представления слов, их загрузка может занять некоторое время!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [12]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Мы также можем извлечь векторное представление из слова, чтобы использовать его при обучении модели классификации. Представление имеет 300 компонентов, но здесь мы показываем только первые 20 компонентов вектора для ясности:


In [13]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Замечательная особенность семантических встраиваний заключается в том, что можно манипулировать векторным кодированием на основе семантики. Например, мы можем попросить найти слово, чье векторное представление максимально близко к словам *король* и *женщина*, и максимально далеко от слова *мужчина*:


In [14]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Пример выше использует некоторую внутреннюю магию GenSym, но основная логика на самом деле довольно проста. Интересная вещь в эмбеддингах заключается в том, что вы можете выполнять обычные операции с векторами на векторах эмбеддинга, и это будет отражать операции с **значениями** слов. Пример выше можно выразить в терминах операций с векторами: мы вычисляем вектор, соответствующий **KING-MAN+WOMAN** (операции `+` и `-` выполняются на векторных представлениях соответствующих слов), а затем находим ближайшее слово в словаре к этому вектору:


In [15]:
# get the vector corresponding to kind-man+woman
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# find the index of the closest embedding vector 
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# find the corresponding word
w2v.index_to_key[min_idx]

'queen'

> **NOTE**: Мы добавили небольшие коэффициенты к векторам *man* и *woman* — попробуйте убрать их, чтобы увидеть, что произойдет.

Чтобы найти ближайший вектор, мы используем инструменты TensorFlow для вычисления вектора расстояний между нашим вектором и всеми векторами в словаре, а затем находим индекс минимального слова с помощью `argmin`.


Хотя Word2Vec кажется отличным способом выражения семантики слов, у него есть множество недостатков, включая следующие:

* Модели CBoW и skip-gram являются **предсказательными векторами**, и они учитывают только локальный контекст. Word2Vec не использует глобальный контекст.
* Word2Vec не учитывает **морфологию** слов, то есть тот факт, что значение слова может зависеть от различных частей слова, таких как корень.

**FastText** пытается преодолеть второе ограничение и расширяет Word2Vec, обучая векторные представления для каждого слова и n-грамм символов, найденных внутри каждого слова. Значения этих представлений затем усредняются в один вектор на каждом этапе обучения. Хотя это добавляет значительное количество вычислений на этапе предварительного обучения, это позволяет векторным представлениям слов кодировать информацию о частях слова.

Другой метод, **GloVe**, использует иной подход к созданию векторных представлений слов, основанный на факторизации матрицы "слово-контекст". Сначала он строит большую матрицу, которая подсчитывает количество появлений слов в различных контекстах, а затем пытается представить эту матрицу в меньших измерениях таким образом, чтобы минимизировать потери при восстановлении.

Библиотека gensim поддерживает эти методы создания векторных представлений слов, и вы можете экспериментировать с ними, изменяя код загрузки модели выше.


## Использование предварительно обученных эмбеддингов в Keras

Мы можем изменить приведённый выше пример, чтобы заполнить матрицу в нашем слое эмбеддинга семантическими эмбеддингами, такими как Word2Vec. Словари предварительно обученного эмбеддинга и текстового корпуса, скорее всего, не будут совпадать, поэтому нам нужно выбрать один из них. Здесь мы рассмотрим два возможных варианта: использование словаря токенизатора и использование словаря из эмбеддингов Word2Vec.

### Использование словаря токенизатора

При использовании словаря токенизатора некоторые слова из словаря будут иметь соответствующие эмбеддинги Word2Vec, а некоторые будут отсутствовать. Учитывая, что размер нашего словаря равен `vocab_size`, а длина вектора эмбеддинга Word2Vec равна `embed_size`, слой эмбеддинга будет представлен весовой матрицей формы `vocab_size`$\times$`embed_size`. Мы заполним эту матрицу, проходя по словарю:


In [9]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


Для слов, которые отсутствуют в словаре Word2Vec, мы можем либо оставить их как нули, либо сгенерировать случайный вектор.

Теперь мы можем определить слой эмбеддинга с предварительно обученными весами:


In [10]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 [==============================] - 10s 10ms/step - loss: 1.1075 - acc: 0.7822 - val_loss: 0.9134 - val_acc: 0.8175


> **Note**: Обратите внимание, что мы устанавливаем `trainable=False` при создании `Embedding`, что означает, что слой Embedding не будет переобучаться. Это может немного снизить точность, но ускоряет процесс обучения.

### Использование словаря для эмбеддингов

Одна из проблем предыдущего подхода заключается в том, что словари, используемые в TextVectorization и Embedding, отличаются. Чтобы решить эту проблему, мы можем воспользоваться одним из следующих решений:
* Переобучить модель Word2Vec на нашем словаре.
* Загрузить наш набор данных, используя словарь из предварительно обученной модели Word2Vec. Словари, используемые для загрузки набора данных, можно указать во время загрузки.

Второй подход кажется проще, поэтому давайте его реализуем. Прежде всего, мы создадим слой `TextVectorization` с указанным словарем, взятым из эмбеддингов Word2Vec:


In [12]:
vocab = list(w2v.vocab.keys())
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

Библиотека word embeddings gensim содержит удобную функцию `get_keras_embeddings`, которая автоматически создаст соответствующий слой embeddings для Keras.


In [13]:
model = keras.models.Sequential([
    vectorizer, 
    w2v.get_keras_embedding(train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Epoch 1/5
938/938 [==============================] - 20s 14ms/step - loss: 1.3377 - acc: 0.4978 - val_loss: 1.2995 - val_acc: 0.5647
Epoch 2/5
938/938 [==============================] - 10s 10ms/step - loss: 1.2587 - acc: 0.5722 - val_loss: 1.2339 - val_acc: 0.5842
Epoch 3/5
938/938 [==============================] - 10s 10ms/step - loss: 1.1980 - acc: 0.5884 - val_loss: 1.1826 - val_acc: 0.5954
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 1.1503 - acc: 0.6002 - val_loss: 1.1417 - val_acc: 0.6018
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 1.1120 - acc: 0.6097 - val_loss: 1.1083 - val_acc: 0.6104


Одна из причин, по которой мы не наблюдаем более высокой точности, заключается в том, что некоторые слова из нашего набора данных отсутствуют в предварительно обученном словаре GloVe, и поэтому они фактически игнорируются. Чтобы преодолеть это, мы можем обучить собственные эмбеддинги на основе нашего набора данных.


## Контекстуальные эмбеддинги

Одним из ключевых ограничений традиционных предварительно обученных представлений эмбеддингов, таких как Word2Vec, является то, что, хотя они и могут захватывать некоторое значение слова, они не способны различать его разные значения. Это может вызывать проблемы в последующих моделях.

Например, слово «play» имеет разные значения в следующих предложениях:
- Я сходил на **пьесу** в театр.
- Джон хочет **поиграть** со своими друзьями.

Предварительно обученные эмбеддинги, о которых мы говорили, представляют оба значения слова «play» одним и тем же эмбеддингом. Чтобы преодолеть это ограничение, необходимо строить эмбеддинги на основе **языковой модели**, которая обучена на большом корпусе текста и *знает*, как слова могут сочетаться в разных контекстах. Обсуждение контекстуальных эмбеддингов выходит за рамки данного урока, но мы вернемся к ним, когда будем говорить о языковых моделях в следующем разделе.



---

**Отказ от ответственности**:  
Этот документ был переведен с использованием сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Несмотря на наши усилия обеспечить точность, автоматические переводы могут содержать ошибки или неточности. Оригинальный документ на его исходном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется профессиональный перевод человеком. Мы не несем ответственности за любые недоразумения или неправильные интерпретации, возникшие в результате использования данного перевода.
